In [ ]:
# Bibliotecas usadas
import csv
import math
import random
import copy

import folium
from folium.plugins import MarkerCluster
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

from selenium import webdriver
import time
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

### Importando dados

In [ ]:
# Caminho para o arquivo enviado
file_path = "dados_csv/Postos Candidatos - São Carlos.csv"

# Inicializando o dicionário
dados_postos_candidatos = {}

with open(file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if len(row) >= 3:
            nome = row[0].strip()
            latitude = row[1].strip()
            longitude = row[2].strip()
            local = row[3].strip()
            
            dados_postos_candidatos[nome] = {
                "latitude": latitude,
                "longitude": longitude,
                "local": local
            }

dados_postos_candidatos

In [ ]:
postos_candidatos = []
postos_candidatos = list(dados_postos_candidatos.keys())

In [ ]:
# Caminho para o arquivo enviado
file_path = "dados_csv/Clientes - São Carlos.csv"

# Inicializando o dicionário
dados_clientes = {}

with open(file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if len(row) >= 2:
            nome = row[0].strip()
            latitude = row[1].strip()
            longitude = row[2].strip()
            
            dados_clientes[nome] = {
                "latitude": latitude,
                "longitude": longitude
            }

dados_clientes

In [ ]:
clientes = []
clientes = list(dados_clientes.keys())

# DISTÂNCIAS RETAS

### Transformação de coordenadas geográficas em euclidianas

In [ ]:
# Função converte latitude/longitude em graus, minutos e segundos
def conversao(x):
    graus = int(x) # se preferir: strip(str(x))
    y = (x - graus) * 60
    minutos =  int(y)
    segundos = (y - minutos) * 60

    return graus, minutos, segundos

# Função calcula a distância latitudinal/longitudinal de dois pontos
def distancia(graus_a, graus_b, minutos_a, minutos_b, segundos_a, segundos_b):
    return ((graus_a - graus_b) * 60) + ((minutos_a - minutos_b) * 1) + ((segundos_a - segundos_b) / 60)
    #return ((-graus_a - (-graus_b)) * 60) + ((-minutos_a - (-minutos_b)) * 1) + ((-segundos_a - (-segundos_b)) / 60)


# Função converte a distância de coordenadas geográficas em distância em milhas
def dist_milha(distancia):
    return abs(distancia * 1852)

# Função calcula a distância euclidiana entre dois pontos
def dist_euclidiana(latitude, longitude):
    return f"{(math.sqrt((latitude**2 + longitude**2))):.2f}" # em metros

### Gerando um mapa com os pontos

In [ ]:
lat_centro = sum([float(dados_clientes[c]["latitude"]) for c in clientes]) / len(clientes)
lon_centro = sum([float(dados_clientes[c]["longitude"]) for c in clientes]) / len(clientes)
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=12)

# Adicionando os marcadores com ícone personalizado
for nome, info in dados_postos_candidatos.items():
    lat = float(info['latitude'])
    lon = float(info['longitude'])
    
    folium.Marker(
        [lat, lon],
        popup=nome,
        icon=folium.Icon(color='orange', icon='bolt', prefix='fa')  # 'bolt' usa ícone tipo raio
    ).add_to(mapa)

for nome, info in dados_clientes.items():
    lat = float(info['latitude'])
    lon = float(info['longitude'])
    
    folium.Marker(
        [lat, lon],
        popup=nome,
        icon=folium.Icon(color='blue', icon='car', prefix='fa')  # 'car' usa ícone de carro
    ).add_to(mapa)

mapa.save("mapas/postos_candidatos_clientes.html")
print("Mapa salvo com sucesso")

In [ ]:
'''
    d_{i,j} = distância euclidiana do cliente i ao posto j
'''

#d = [[0 for j in range(len(postos_candidatos))] for i in range(len(clientes))]
d = [[0 for j in range(len(postos_candidatos))] for i in range(len(clientes))] # d_ij

# Conversão das coordenadas geográficas em euclidianas
for i, x in enumerate(clientes):
    for j, y in enumerate(postos_candidatos):
        graus_lat_a, minutos_lat_a, segundos_lat_a = conversao(float(dados_clientes[x]['latitude']))
        graus_lon_a, minutos_lon_a, segundos_lon_a = conversao(float(dados_clientes[x]['longitude']))
        graus_lat_b, minutos_lat_b, segundos_lat_b = conversao(float(dados_postos_candidatos[y]['latitude']))
        graus_lon_b, minutos_lon_b, segundos_lon_b = conversao(float(dados_postos_candidatos[y]['longitude']))

        dist_lat = distancia(graus_lat_a, graus_lat_b, minutos_lat_a, minutos_lat_b, segundos_lat_a, segundos_lat_b)
        dist_lon = distancia(graus_lon_a, graus_lon_b, minutos_lon_a, minutos_lon_b, segundos_lon_a, segundos_lon_b)
        
        dist_lat_milha = dist_milha(dist_lat)
        dist_lon_milha = dist_milha(dist_lon)
        
        d[i][j] = dist_euclidiana(dist_lat_milha, dist_lon_milha) # Matriz de distâncias


### Modelagem do Problema

In [ ]:
M = len(clientes)
N = len(postos_candidatos)

# Criação do modelo
m = gp.Model("Problema de Localização de Facilidades")

# Variáveis de decisão contínuas: x[i, j] representa o quanto o cliente i está atendido pelo posto j
x = m.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name="Facilidade")

# Variáveis binárias: y[j] = 1 se o posto j for escolhido, 0 caso contrário
y = m.addVars(N, vtype=GRB.BINARY, name="Aberto")

# Função objetivo: minimizar a distância total ponderada
m.setObjective(gp.quicksum(d[i][j] * x[i, j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

# Restrição 1: cada cliente deve ser atendido exatamente por um posto
m.addConstrs((gp.quicksum(x[i, j] for j in range(N)) == 1 for i in range(M)), name="DemandaClientes")

# Restrição 2: número de instalações limitadas (opcional, aqui assumindo exatamente K postos a abrir)
P = 5  # por exemplo, limite de instalação de 3 postos dados 10 postos candidatos
m.addConstr(gp.quicksum(y[j] for j in range(N)) == P, name="NumeroInstalacoes")

# Restrição 3: cliente só pode ser atendido por um posto se ele estiver aberto
m.addConstrs((x[i, j] <= y[j] for i in range(M) for j in range(N)), name="ClientePorPosto")

# Otimização
start_time = time.time()
m.optimize()
execution_time = time.time() - start_time

In [ ]:
clientes = list(dados_clientes.keys())

#print("Distância total mínima: "+str(m.objVal))
print("Distância total mínima: "+f"{float(m.objVal):.2f}")

print("\t                                       Postos")
print("Clientes   \tA\tB\tC\tD\tE\tF\tG\tH\tI\tJ")

for i in range(M):
  print(clientes[i], end = '')
  for j in range(N):
    print("\t"+str(round(x[i,j].X)), end = '')
  print()

print('\n')

print('Tempo de Execução: ', execution_time)
print("Custo mínimo total: "+str(m.objVal))

print('Solução Otima: [', end='')
for j in range(N):
    if j < N - 1:
        print(str(round(y[j].X)) + ', ', end='')
    else:
        print(str(round(y[j].X)) + ']')

fornecedores_abertos = [j for j in range(N) if y[j].X > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)

### Plotando a solução no mapa

In [ ]:
# Criando o mapa base
lat_centro = sum([float(dados_clientes[c]["latitude"]) for c in clientes]) / M     #M = len(clientes)
lon_centro = sum([float(dados_clientes[c]["longitude"]) for c in clientes]) / M
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=12)
cluster = MarkerCluster().add_to(mapa)

# Adiciona marcadores dos postos instalados
for j in range(N):
    #if y[j].X > 0.5:
    if y[j].X <= 1: # Adiciona marcadores em todos os postos
        folium.Marker(
            location=[float(dados_postos_candidatos[postos_candidatos[j]]["latitude"]), float(dados_postos_candidatos[postos_candidatos[j]]["longitude"])],
            popup=f"Posto: {postos_candidatos[j]}",
            icon=folium.Icon(color="green", icon="bolt", prefix="fa")
        ).add_to(mapa)

# Adiciona clientes e linhas de conexão
for i in range(M):
    cliente_info = dados_clientes[clientes[i]]
    folium.Marker(
        location=[cliente_info["latitude"], cliente_info["longitude"]],
        popup=f"Cliente: {clientes[i]}",
        icon=folium.Icon(color="blue", icon="user", prefix="fa")
    ).add_to(cluster)

    # Descobre qual posto o atende (com x[i,j] > 0.5)
    for j in range(N):
        if x[i,j].X > 0.5:
            posto_candidatos_info = dados_postos_candidatos[postos_candidatos[j]]
            folium.PolyLine(
                locations=[
                    [float(cliente_info["latitude"]), float(cliente_info["longitude"])],
                    [float(posto_candidatos_info["latitude"]), float(posto_candidatos_info["longitude"])]
                ],
                color='black',
                weight=3,
                opacity=1
            ).add_to(mapa)
            break  # um cliente é atendido por apenas um posto

# Salva o mapa
mapa.save("mapas/solucao_postos_candidatos_distancia_euclidiana.html")
print("Mapa salvo com sucesso")

## VND

In [ ]:
# Função que resolve o modelo dado um vetor binário de postos abertos
def f(y_):
    #start_time = time.time()
    m = gp.Model()
    m.setParam('OutputFlag', 0)

    x = m.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0)

    m.setObjective(gp.quicksum(d[i][j] * x[i, j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

    for i in range(M):
        m.addConstr(gp.quicksum(x[i, j] for j in range(N)) == 1)
        for j in range(N):
            m.addConstr(x[i, j] <= y_[j])

    m.addConstr(gp.quicksum(y_[j] for j in range(N)) == P)

    #m.setParam('TimeLimit', 60) # Limite de tempo de execução do modelo
    m.optimize()

    #end_time = time.time()
    #execution_time = end_time - start_time
    
    # Imprimir o tempo de execução
    #print(f"Tempo de execução: {execution_time} segundos")
    return m.objVal


# Vizinhança Swap: troca entre postos aberto e fechado (swap entre 1 e 0)
def vizinhanca_v1(y_):
    vizinhos = []
    abertos = [i for i, val in enumerate(y_) if val == 1]
    fechados = [i for i, val in enumerate(y_) if val == 0]
    for i in abertos:
        for j in fechados:
            novo = y_.copy()
            novo[i], novo[j] = 0, 1
            vizinhos.append(novo)
    return vizinhos

# Vizinhança olha posições antecessora e sucessora da solução para possível troca
def vizinhanca_v3(y_):
    vizinhos = []
    n = len(y_)
    
    for i in range(n):
        # Tenta trocar com a posição anterior
        if i > 0 and y_[i] != y_[i - 1]:
            novo = y_.copy()
            novo[i], novo[i - 1] = novo[i - 1], novo[i]
            vizinhos.append(novo)
        # Se não puder trocar com anterior, tenta sucessor
        elif i < n - 1 and y_[i] != y_[i + 1]:
            novo = y_.copy()
            novo[i], novo[i + 1] = novo[i + 1], novo[i]
            vizinhos.append(novo)
        # Caso contrário, não adiciona vizinho

    return vizinhos

def vizinhanca_v7(y_):
    vizinhos = []
    abertos = [i for i, val in enumerate(y_) if val == 1]
    fechados = [i for i, val in enumerate(y_) if val == 0]

    for i in abertos:
        if not fechados:
            break  # não há para onde trocar
        j = random.choice(fechados)
        novo = y_.copy()
        novo[i], novo[j] = 0, 1
        vizinhos.append(novo)
        fechados.remove(j)  # evita usar o mesmo 0 novamente (opcional)

    return vizinhos

# vizinhanca troca o primeiro posto aberto pelos len(y_)/sum(y_) ultimos postos fechados
def vizinhanca_v9(y_):
    vizinhos = []
    primeiro_aberto = next((i for i, val in enumerate(y_) if val == 1), None)
    
    if primeiro_aberto is None:
        return []  # Não há postos abertos para trocar

    for j in reversed(range(len(y_))):
        if y_[j] == 0:
            novo = y_.copy()
            novo[primeiro_aberto] = 0
            novo[j] = 1
            vizinhos.append(novo)

    return vizinhos
# Metaheurística VND
def VND(s0, vizinhancas):
    tempo_max = 1800 # Tempo máximo de execucao: 30 minutos
    tempo_limite = 150  # Limite de tempo de analise em cada vizinhanca: 2,5 minutos
    s = s0
    f_s = f(s)  # Valor inicial da função objetivo (limitante dual?)
    k = 0

    inicio = time.time()
    while k < len(vizinhancas) or inicio < tempo_max:
        start_time = time.time()
        vizinhos = vizinhancas[k](s)  # vizinhos de s
        melhor = s
        f_melhor = f_s

        print(f"Tamanho da vizinhança {k}: {len(vizinhos)}")

        encontrou_melhor = False

        for v in vizinhos:
            if time.time() - start_time > tempo_limite:
                print(f"Tempo excedido na vizinhança {k}")
                encontrou_melhor = False # Não encontrou solução melhor na vizinhança
                break

            f_v = f(v)
            if f_v < f_melhor:
                melhor = v
                f_melhor = f_v
                encontrou_melhor = True # Encontrou solução melhor na vizinhança
                break

        if encontrou_melhor:
            s = melhor
            f_s = f_melhor
            k = 0  # Volta para a primeira estrutura de vizinhança
        else:
            k += 1  # Passa para a próxima estrutura de vizinhança

    return s, f_s

# Estratégia gulosa para gerar solução inicial
def metodo_guloso(m, n, P, d):
    aberto = []
    resto = list(range(n))
    
    # Inicializando o vetor de alocação com distâncias infinitas (para cada cliente)
    custo_atribuicao = [float('inf')] * m
    
    for _ in range(P):
        melhor_facilidade = None
        melhor_melhoria = float('inf')
        
        for j in resto:
            custo_total = 0
            for i in range(m):  # Para cada cliente
                # Considera a menor distância entre o já atribuído e o novo fornecedor j
                custo_total += min(custo_atribuicao[i], d[i][j])
            if custo_total < melhor_melhoria:
                melhor_melhoria = custo_total
                melhor_facilidade = j
        
        aberto.append(melhor_facilidade)
        resto.remove(melhor_facilidade)

        # Atualiza os custos de alocação dos clientes com o novo fornecedor aberto
        for i in range(m):
            custo_atribuicao[i] = min(custo_atribuicao[i], d[i][melhor_facilidade])

    # Cria vetor solução: 1 se fornecedor está aberto, 0 caso contrário
    s0 = [1 if j in aberto else 0 for j in range(n)]
    return s0

#### SOLUÇÃO ALEATÓRIA

In [ ]:
# Execução
posicoes = random.sample(range(N), P)
s0 = [1 if i in posicoes else 0 for i in range(N)]

fornecedor_inicial = [j for j in range(N) if s0[j] > 0.5]
print("Fornecedores inicial:", fornecedor_inicial)

vizinhancas = [vizinhanca_v9, vizinhanca_v7, vizinhanca_v3, vizinhanca_v1]

start_time = time.time()
melhor_solucao, custo = VND(s0, vizinhancas)
execution_time = time.time() - start_time

print('Tempo de Execução VND: ', execution_time)
print("Melhor solução:", melhor_solucao)
print("Custo mínimo total:", custo)

fornecedores_abertos = [j for j in range(N) if melhor_solucao[j] > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)

#### ESTRATÉGIA GULOSA

In [ ]:
# estrategia gulosa
distancia = d.copy()
distancia = [[float(valor) for valor in linha] for linha in distancia]

start_time = time.time()
s0 = metodo_guloso(M, N, P, distancia)
print(f"Tempo de execução Método Guloso: {time.time() - start_time}")

fornecedores_abertos_guloso = [j for j in range(N) if s0[j] > 0.5]
print("Fornecedores abertos guloso:", fornecedores_abertos_guloso,'\n')

vizinhancas = [vizinhanca_v9, vizinhanca_v7, vizinhanca_v3, vizinhanca_v1]

start_time = time.time()
melhor_solucao, custo = VND(s0, vizinhancas)
execution_time = time.time() - start_time

print('Tempo de Execução VND: ', execution_time)
print("Melhor solução:", melhor_solucao)
print("Custo mínimo total:", custo)
fornecedores_abertos = [j for j in range(N) if melhor_solucao[j] > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)

# DISTÂNCIAS REAIS

##### Trajeto mais rápido levando em consideração as condições de trânsito

#### Automatização do cálculo da melhor distância de um cliente até um posto

In [ ]:
def info():
    sleep(3)
    distance = driver.find_element('xpath', '//*[@id="section-directions-trip-0"]/div[1]/div/div[1]/div[2]/div').text # armazena a distancia entre um cliente e um posto
    if 'km' in distance: # Se estiver em km, converter para metros
        distancia = float(distance.replace(' km', '').replace(',', '.'))
        distancia = distancia * 1000
    elif 'm' in distance: # Se já estiver em metros, apenas extrair o número
        distancia = float(distance.replace(' m', '').replace(',', '.'))
    else:
        raise ValueError(f"Formato de distância desconhecido: {distance}")

    return distancia

def search(coord_posto, coord_cliente):
    sleep(2)
    driver.find_element('xpath', '//*[@id="hArJGc"]/span').click() # clica em rota
    sleep(1)
    driver.find_element('xpath', '//*[@id="sb_ifc51"]/input').send_keys(coord_posto) # posto
    sleep(2)
    driver.find_element('xpath', '//*[@id="sb_ifc50"]/input').click()
    driver.find_element('xpath', '//*[@id="sb_ifc50"]/input').send_keys(coord_cliente) # cliente
    sleep(1)
    driver.find_element('xpath', '//*[@id="directions-searchbox-0"]/button[1]/span').click() # clica em pesquisar
    sleep(1)
    driver.find_element('xpath', '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[2]/button/div[1]/span[3]').click() # rotas por carro

In [ ]:
'''
    d_{i,j} = distância euclidiana do cliente i ao posto j
'''

d = [[0 for j in range(len(postos_candidatos))] for i in range(len(clientes))] #d_ij

inicio = time.time()
for i, c in enumerate(clientes):
    for j, p in enumerate(postos_candidatos):
        coord_cliente = {}
        coord_posto = {}
        
        coord_posto = f"{dados_postos_candidatos[p]["latitude"]}, {dados_postos_candidatos[p]["longitude"]}"
        coord_cliente = f"{dados_clientes[c]["latitude"]}, {dados_clientes[c]["longitude"]}"

        driver = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=driver)
        #driver.get('https://www.google.com/maps/@-21.9552607,-48.0116736,15z/data=!5m1!1e1?entry=ttu&g_ep=EgoyMDI1MDQxNi4xIKXMDSoJLDEwMjExNDU1SAFQAw%3D%3D')
        driver.get('https://www.google.com/maps/')
        sleep(2)

        search(coord_posto, coord_cliente)
        d[i][j] = info()

        driver.quit() # encerra o navegador
        
fim = time.time()
print(fim - inicio)

In [ ]:
# Distâncias do dia 07/06/2025 - parte da tarde
#d = [[8800.0, 11000.0, 6200.0, 3600.0, 11100.0, 3400.0, 2900.0, 12100.0, 15400.0, 2100.0], [6000.0, 8200.0, 3400.0, 6800.0, 8200.0, 500.0, 3200.0, 8000.0, 10700.0, 5100.0], [5600.0, 8900.0, 7900.0, 12800.0, 9200.0, 10700.0, 9400.0, 4400.0, 7600.0, 10600.0], [6800.0, 3900.0, 5800.0, 19000.0, 5300.0, 6900.0, 9600.0, 7800.0, 7800.0, 11500.0], [2400.0, 4800.0, 2500.0, 9700.0, 4600.0, 3400.0, 4200.0, 4400.0, 7100.0, 8000.0], [4300.0, 7200.0, 550.0, 6900.0, 6100.0, 3200.0, 2000.0, 3800.0, 8500.0, 5300.0], [13100.0, 16400.0, 7600.0, 9100.0, 16700.0, 8700.0, 7400.0, 11900.0, 15100.0, 8900.0], [6400.0, 450.0, 7300.0, 18600.0, 4900.0, 8700.0, 9500.0, 7400.0, 7400.0, 13200.0], [10800.0, 14100.0, 6500.0, 8200.0, 14400.0, 8100.0, 6700.0, 9600.0, 12800.0, 8200.0], [7200.0, 14000.0, 3900.0, 5200.0, 8900.0, 3000.0, 1600.0, 9600.0, 12800.0, 3600.0], [11600.0, 14800.0, 6300.0, 6800.0, 15100.0, 6300.0, 4900.0, 10400.0, 13600.0, 6300.0], [14700.0, 17900.0, 9500.0, 1300.0, 18300.0, 6100.0, 5300.0, 13500.0, 16700.0, 1600.0], [6900.0, 10200.0, 3300.0, 8500.0, 10500.0, 6200.0, 3700.0, 5700.0, 8900.0, 6900.0], [5000.0, 11900.0, 2400.0, 7200.0, 12200.0, 4900.0, 2800.0, 7400.0, 10700.0, 5500.0], [2900.0, 4700.0, 5600.0, 15200.0, 5000.0, 7900.0, 7800.0, 2100.0, 3900.0, 13500.0], [6400.0, 8300.0, 9100.0, 18300.0, 8600.0, 11400.0, 11300.0, 5600.0, 5900.0, 16800.0], [7000.0, 1100.0, 7900.0, 15500.0, 5600.0, 9300.0, 10100.0, 8000.0, 8000.0, 13900.0], [4800.0, 7000.0, 4500.0, 9800.0, 7000.0, 3600.0, 6300.0, 6800.0, 9500.0, 8200.0], [20900.0, 24100.0, 15700.0, 7900.0, 24500.0, 13700.0, 14200.0, 19700.0, 22900.0, 9100.0], [1900.0, 3400.0, 3100.0, 10800.0, 5000.0, 4500.0, 5300.0, 4600.0, 7500.0, 9100.0], [3300.0, 2500.0, 6300.0, 14800.0, 1800.0, 7500.0, 8500.0, 4300.0, 4300.0, 15300.0], [3900.0, 6100.0, 3600.0, 9600.0, 6200.0, 3300.0, 6000.0, 6000.0, 8600.0, 7900.0], [4300.0, 6200.0, 6100.0, 14900.0, 6500.0, 9500.0, 8300.0, 2600.0, 600.0, 16100.000000000002], [10300.0, 13600.0, 4900.0, 5500.0, 13900.0, 4000.0, 2600.0, 9100.0, 12300.0, 3800.0], [12300.0, 15600.0, 5000.0, 3900.0, 15900.0, 4400.0, 3500.0, 11100.0, 14400.0, 2300.0]]

### Modelagem Matemática

In [ ]:
M = len(clientes)
N = len(postos_candidatos)

# Criação do modelo
m = gp.Model("Problema de Localização de Facilidades")

# Variáveis de decisão contínuas: x[i, j] representa o quanto o cliente i está atendido pelo posto j
x = m.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name="Facilidade")

# Variáveis binárias: y[j] = 1 se o posto j for escolhido, 0 caso contrário
y = m.addVars(N, vtype=GRB.BINARY, name="Aberto")

# Função objetivo: minimizar a distância total ponderada
m.setObjective(gp.quicksum(d[i][j] * x[i, j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

# Restrição 1: cada cliente deve ser atendido exatamente por um posto
m.addConstrs((gp.quicksum(x[i, j] for j in range(N)) == 1 for i in range(M)), name="DemandaClientes")

# Restrição 2: número de instalações limitadas (opcional, aqui assumindo exatamente K postos a abrir)
P = 5  # por exemplo, limite de instalação de 3 postos dados 10 postos candidatos
m.addConstr(gp.quicksum(y[j] for j in range(N)) == P, name="NumeroInstalacoes")

# Restrição 3: cliente só pode ser atendido por um posto se ele estiver aberto
m.addConstrs((x[i, j] <= y[j] for i in range(M) for j in range(N)), name="ClientePorPosto")

# Otimização
m.optimize()

In [ ]:
print('Tempo de Execução: ', execution_time)
print("Custo mínimo total: "+str(m.objVal))

print('Solução Otima: [', end='')
for j in range(N):
    if j < N - 1:
        print(str(round(y[j].X)) + ', ', end='')
    else:
        print(str(round(y[j].X)) + ']')

fornecedores_abertos = [j for j in range(N) if y[j].X > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)

In [ ]:
# Criando o mapa base
lat_centro = sum([float(dados_clientes[c]["latitude"]) for c in clientes]) / M     #M = len(clientes)
lon_centro = sum([float(dados_clientes[c]["longitude"]) for c in clientes]) / M
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=12)
cluster = MarkerCluster().add_to(mapa)

# Adiciona marcadores dos postos instalados
for j in range(N):
    #if y[j].X > 0.5:
    if y[j].X <= 1: # Adiciona marcadores em todos os postos
        folium.Marker(
            location=[float(dados_postos_candidatos[postos_candidatos[j]]["latitude"]), float(dados_postos_candidatos[postos_candidatos[j]]["longitude"])],
            popup=f"Posto: {postos_candidatos[j]}",
            icon=folium.Icon(color="green", icon="bolt", prefix="fa")
        ).add_to(mapa)

# Adiciona clientes e linhas de conexão
for i in range(M):
    cliente_info = dados_clientes[clientes[i]]
    folium.Marker(
        location=[cliente_info["latitude"], cliente_info["longitude"]],
        popup=f"Cliente: {clientes[i]}",
        icon=folium.Icon(color="blue", icon="user", prefix="fa")
    ).add_to(cluster)

    # Descobre qual posto o atende (com x[i,j] > 0.5)
    for j in range(N):
        if x[i,j].X > 0.5:
            posto_candidatos_info = dados_postos_candidatos[postos_candidatos[j]]
            folium.PolyLine(
                locations=[
                    [float(cliente_info["latitude"]), float(cliente_info["longitude"])],
                    [float(posto_candidatos_info["latitude"]), float(posto_candidatos_info["longitude"])]
                ],
                color='black',
                weight=3,
                opacity=1
            ).add_to(mapa)
            break  # um cliente é atendido por apenas um posto

# Salva o mapa
mapa.save("mapas/solucao_postos_candidatos_distancias_reais.html")
print("Mapa salvo com sucesso")

## VND

#### SOLUÇÃO ALEATÓRIA

In [ ]:
# Execução
posicoes = random.sample(range(N), P)
s0 = [1 if i in posicoes else 0 for i in range(N)]

fornecedores_abertos = [j for j in range(N) if s0[j] > 0.5]
print("Fornecedores iniciais abertos:", fornecedores_abertos)

vizinhancas = [vizinhanca_v9, vizinhanca_v7, vizinhanca_v3, vizinhanca_v1]

start_time = time.time()
melhor_solucao, custo = VND(s0, vizinhancas)
execution_time = time.time() - start_time

print('Tempo de Execução VND: ', execution_time)
print("Melhor solução:", melhor_solucao)
print("Custo mínimo total:", custo)

fornecedores_abertos = [j for j in range(N) if melhor_solucao[j] > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)

#### ESTRATÉGIA GULOSA

In [ ]:
# estrategia gulosa
distancia = d.copy()
distancia = [[float(valor) for valor in linha] for linha in distancia]

start_time = time.time()
s0 = metodo_guloso(M, N, P, distancia)
print(f"Tempo de execução Método Guloso: {time.time() - start_time}")

fornecedores_abertos_guloso = [j for j in range(N) if s0[j] > 0.5]
print("Fornecedores abertos guloso:", fornecedores_abertos_guloso,'\n')

vizinhancas = [vizinhanca_v9, vizinhanca_v7, vizinhanca_v3, vizinhanca_v1]

start_time = time.time()
melhor_solucao, custo = VND(s0, vizinhancas)
execution_time = time.time() - start_time
print('Tempo de Execução VND: ', execution_time)

print("Melhor solução:", melhor_solucao)
print("Custo mínimo total:", custo)


fornecedores_abertos = [j for j in range(N) if melhor_solucao[j] > 0.5]
print("Fornecedores abertos:", fornecedores_abertos)